In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.linear_model import LogisticRegression
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import pickle
import torch


In [ ]:
with open('../data/graphs/5/linegraph_tg.pkl', 'rb') as f:
    graph_data = pickle.load(f)

bins = [int(i) for i in '400 800 1300 2100 3000 3700 4700 7020 9660'.split()]
# bins = [int(i) for i in '3000'.split()]

bins = torch.tensor(bins)


In [ ]:
y = graph_data.y[graph_data.y > 0]
x = graph_data.x[graph_data.y > 0].numpy()
# x = x[:, 5].reshape(-1, 1)

y = torch.bucketize(y, boundaries=bins).numpy()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
print(f"X_train shape: {X_train.shape}")
print(f"X_valid shape: {X_valid.shape}")
print(f"X_test shape: {X_test.shape}")


In [ ]:
# Create a Logistic Regression model
model = LogisticRegression(max_iter=10000, solver='saga', random_state=42, multi_class='auto')
# Fit the model on the training data
model.fit(X_train, y_train)
# Evaluate the model on the validation data
valid_accuracy = model.score(X_valid, y_valid)

print(confusion_matrix(y_valid, model.predict(X_valid)))
print(f"Validation accuracy: {valid_accuracy:.4f}")

print(classification_report(y_valid, model.predict(X_valid), zero_division=0))
